In [2]:
import torch
import time
import cv2
import matplotlib as plt
import gym
from Gridworld_Game import Gridworld

In [3]:
GRID_SIZE = 4
env = Gridworld(Grid_size=GRID_SIZE, mode='static')

In [4]:
env.observation_space

Box(0, 1, (4, 4, 4), int32)

In [5]:
env.actions_space

Discrete(4)

In [6]:
action = env.actions_space.sample()
action

3

In [7]:
env.reset()
env.render(mode='human')
time.sleep(10)
env.close()

In [15]:
env.step(action)
env.reset()
env.render(mode='human')
time.sleep(10)
env.close()

In [8]:
actions = {'left': 0, 'right': 1, 'up': 2, 'down': 3}

In [9]:
env.step(actions['right'])

(array([[[0, 0, 0, 0],
         [1, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],
 
        [[0, 0, 0, 0],
         [0, 1, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],
 
        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],
 
        [[0, 0, 0, 0],
         [0, 0, 0, 1],
         [0, 0, 0, 0],
         [0, 0, 1, 0]]]),
 -1,
 False,
 {'items': {'Agent': 0, 'Hole': 1, 'Wall': 2, 'Goal': 3},
  'Grid_size': 4,
  'actions': ['Left', 'Right', 'Up', 'Down']})

In [10]:
class Net(torch.nn.Module):
    def __init__(self, input_shape, output_shape):
        super(Net, self).__init__()
        self.linear1 = torch.nn.Linear(input_shape, 150)
        self.linear2 = torch.nn.Linear(150, 100)
        self.linear3 = torch.nn.Linear(100, output_shape)
        self.activation = torch.nn.ReLU()

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.activation(x)
        x = self.linear3(x)
        return x

In [11]:
EPOCHS = 3000
NUM_OBJECTS = 4
INPUT_SHAPE = GRID_SIZE**2 * NUM_OBJECTS
OUTPUT_SHAPE = len(actions)
INPUT_SHAPE

64

In [12]:
Agent = Net(INPUT_SHAPE, OUTPUT_SHAPE)
Agent

Net(
  (linear1): Linear(in_features=64, out_features=150, bias=True)
  (linear2): Linear(in_features=150, out_features=100, bias=True)
  (linear3): Linear(in_features=100, out_features=4, bias=True)
  (activation): ReLU()
)

In [13]:
import numpy as np

In [14]:
def preprocess_state(state, input_shape, add_noise=False):
    # best practice to add noise to the state to avoid overfitting, input_shape is a tuple here
    if add_noise == True:
        state = state.reshape(*input_shape) + np.random.rand(*input_shape)/10.0
    else:
        state = state.reshape(*input_shape)
    return state
        

In [15]:
game = Gridworld(Grid_size=GRID_SIZE, mode='static')
state1 = game.state
state1 = preprocess_state(state1, (1, INPUT_SHAPE), add_noise=True)
state1

array([[1.06286076e+00, 3.06814298e-02, 3.65200698e-02, 1.90195303e-02,
        8.01395299e-02, 9.58418444e-02, 7.91724821e-02, 3.47847091e-02,
        2.57894607e-02, 4.36832826e-02, 3.33497434e-02, 9.59607619e-02,
        4.65884789e-03, 8.43493152e-02, 4.27849577e-02, 7.95197097e-02,
        3.85309131e-02, 6.83272674e-02, 3.16111914e-02, 3.33479278e-02,
        9.88726486e-02, 1.04285153e+00, 1.07561840e-02, 7.79797525e-02,
        7.45414592e-02, 1.18979367e-02, 9.38537281e-02, 5.14081294e-02,
        6.51876622e-02, 4.00064835e-02, 7.28824133e-02, 6.58854834e-02,
        5.80070083e-02, 7.86681620e-03, 8.01854791e-02, 6.47496649e-03,
        9.05098139e-02, 3.11012067e-03, 3.35210421e-02, 8.98892306e-02,
        2.29426731e-02, 5.66499795e-02, 5.95050802e-02, 4.59664178e-02,
        7.63124728e-02, 1.04307271e-04, 4.04869756e-02, 5.34131024e-02,
        4.31086447e-02, 9.22122908e-03, 6.72308911e-03, 1.65777781e-02,
        9.50821603e-02, 3.91474461e-02, 6.53184880e-03, 1.056944

In [16]:
x = torch.Tensor([2.0])
x.detach?

Docstring:
Returns a new Tensor, detached from the current graph.

The result will never require gradient.

This method also affects forward mode AD gradients and the result will never
have forward mode AD gradients.

.. note::

  Returned Tensor shares the same storage with the original one.
  In-place modifications on either of them will be seen, and may trigger
  errors in correctness checks.
  IMPORTANT NOTE: Previously, in-place size / stride / storage changes
  (such as `resize_` / `resize_as_` / `set_` / `transpose_`) to the returned tensor
  also update the original tensor. Now, these in-place changes will not update the
  original tensor anymore, and will instead trigger an error.
  For sparse tensors:
  In-place indices / values changes (such as `zero_` / `copy_` / `add_`) to the
  returned tensor will not update the original tensor anymore, and will instead
  trigger an error.
Type:      builtin_function_or_method

In [17]:
import random

In [18]:
random.random?

Signature: random.random()
Docstring: random() -> x in the interval [0, 1).
Type:      builtin_function_or_method

In [19]:
def get_action(Q_val, num_actions, epsilon):
    if (random.random() < epsilon):
        action = np.random.randint(0, 4)
    else:
        action = np.argmax(Q_val, 1)[0]
    return action
    

In [20]:
# test get_action
epsilon = 1.0

state1 = torch.from_numpy(state1).float()
Q_val = Agent(state1) # returns a tensor with 4 elements
Q_val_numpy = Q_val.detach().numpy()
action = get_action(Q_val_numpy, len(actions), epsilon)
action    

1

In [21]:
Q_val_numpy

array([[ 0.0313306 , -0.09189114,  0.01961648, -0.00459179]],
      dtype=float32)

In [22]:
class target_maker:
    def __init__(self, terminal_rewards):
        self.terminal_rewards = terminal_rewards  # +10 for reaching target, -10 for falling in hole

    def __call__(self, current_reward, gamma, maxQ):
        if current_reward not in self.terminal_rewards: #N
            Y = current_reward + (gamma * maxQ)
        else:
            Y = current_reward
        Y = torch.Tensor([Y]).detach()
        return Y
            

In [24]:
# training loop
gamma = 0.9
terminal_rewards = [-10, +10]
create_target = target_maker(terminal_rewards)
loss_fn = torch.nn.MSELoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(Agent.parameters(), lr=learning_rate)
losses = []

for i in range(EPOCHS):
    game = Gridworld(Grid_size=GRID_SIZE, mode='static')
    state1 = game.state  # state1.shape == (4, 4, 4)
    state1 = preprocess_state(state1, (1, INPUT_SHAPE), add_noise=True)
    state1 = torch.from_numpy(state1).float()
    gate = 1
    while gate == 1:
        Q_val = Agent(state1) # returns a tensor with 4 elements
        Q_val_numpy = Q_val.detach().numpy()
        action = get_action(Q_val_numpy, len(actions), epsilon)
        state2, reward, done, _ = game.step(action)
        state2 = preprocess_state(state2, (1, INPUT_SHAPE), add_noise=True)
        state2 = torch.from_numpy(state2).float()
        game.render()
        with torch.no_grad():
            newQ = Agent(state2)
        maxQ = torch.max(newQ)
        Y = create_target(reward, gamma, maxQ)
        X = Q_val.squeeze()[action]  # previously tensor([[value1, value2, etc..]])
        loss = loss_fn(X, Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        state1 = state2
        losses.append(loss)
        if done == True:
            gate = 0
    if epsilon > 0.1:
        epsilon -= (1/EPOCHS)
    game.close()
        

In [25]:
for i in range(10):
    game = Gridworld(Grid_size=GRID_SIZE, mode='static')
    state1 = game.state
    state1 = preprocess_state(state1, (1, INPUT_SHAPE), add_noise=True)
    state1 = torch.from_numpy(state1).float()
    gate = 1
    while gate == 1:
        Q_val = Agent(state1)
        Q_val_numpy = Q_val.detach().numpy()
        action = np.argmax(Q_val_numpy, 1)[0]
        state2, reward, done, _ = game.step(action)
        state2 = preprocess_state(state2, (1, INPUT_SHAPE), add_noise=True)
        state2 = torch.from_numpy(state2).float()
        game.render()
        time.sleep(1)
        state1 = state2
        if done == True:
            gate = 0
            game.close()
        